## Building A Chatbot
 We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")



In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000018B09DD4250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018B360088E0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Anup and I am a AI Engineer")])

AIMessage(content="Nice to meet you, Anup. As an AI Engineer, you're likely working with various technologies to develop intelligent systems. What areas of AI are you most interested in or have you worked with, such as natural language processing, computer vision, or machine learning?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 48, 'total_tokens': 102, 'completion_time': 0.083982577, 'completion_tokens_details': None, 'prompt_time': 0.002228404, 'prompt_tokens_details': None, 'queue_time': 0.049101416, 'total_time': 0.086210981}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfe14-a6cb-79e2-8be8-e70f8830df14-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 48, 'output_tokens': 54, 'total_tokens': 102})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Anup and I am a AI Engineer"),
        AIMessage(content="Hello Anup! It's nice to meet you. \n\nAs a AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Anup and you're an AI Engineer. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 117, 'total_tokens': 138, 'completion_time': 0.024378126, 'completion_tokens_details': None, 'prompt_time': 0.00644752, 'prompt_tokens_details': None, 'queue_time': 0.05451492, 'total_time': 0.030825646}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfe15-2b8a-7fb1-959d-688849184242-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 117, 'output_tokens': 21, 'total_tokens': 138})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Anup and I am a AI Engineer")],
    config=config
)

In [19]:
response.content

"We just met, and I think there's been a mix-up. Earlier, we started a conversation as if you were Krish, a Chief AI Engineer. Now, it seems like you're introducing yourself as Anup, an AI Engineer. Am I correct in assuming that we're starting a new conversation with you as Anup?"

In [20]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Anup.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 303, 'total_tokens': 310, 'completion_time': 0.012737503, 'completion_tokens_details': None, 'prompt_time': 0.016665332, 'prompt_tokens_details': None, 'queue_time': 0.049280428, 'total_time': 0.029402835}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfe21-446f-7d01-a44e-17d177441b8a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 303, 'output_tokens': 7, 'total_tokens': 310})

In [21]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

In [22]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"You're telling me your name is John again! I've already got that info stored, but feel free to introduce yourself anytime you like."

In [23]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John. I've got that one down!"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [25]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Anup")]})

AIMessage(content="Nice to meet you, Anup. How can I assist you today? Do you have any questions or topics you'd like to discuss?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 57, 'total_tokens': 86, 'completion_time': 0.031979596, 'completion_tokens_details': None, 'prompt_time': 0.003036967, 'prompt_tokens_details': None, 'queue_time': 0.051356021, 'total_time': 0.035016563}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfe29-1788-75d3-9de2-2f59a09a27dd-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 29, 'total_tokens': 86})

In [26]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [27]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Anup")],
    config=config
)

response

AIMessage(content="Hello Anup, it's nice to meet you. Is there something I can help you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 57, 'total_tokens': 79, 'completion_time': 0.028225758, 'completion_tokens_details': None, 'prompt_time': 0.003099546, 'prompt_tokens_details': None, 'queue_time': 0.054703974, 'total_time': 0.031325304}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfe29-663a-7ff3-91eb-0aa91dc93024-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 22, 'total_tokens': 79})

In [28]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Anup. How can I assist you further?'

In [29]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [30]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Anup")],"language":"Nepali"})
response.content

'Anup भाई/दाई, तपाईंले केहि प्रश्न पूछ्नु भयो?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [31]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [33]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Anup")],"language":"Nepali"},
    config=config
)
repsonse.content

'Namaste Anup, कस्तो छ? (Kasto chha?) के हो? (Ke ho?) तपाईंको कति भन्दा छ? (Tapaiñko kitabhandaa chha?)'

In [34]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Nepali"},
    config=config,
)

In [35]:
response.content

'तपाईंको नाम के छ? (Tapaiñko naam ke chha?) तपाईंको नाम अनुप (Anup) छ।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [36]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

e:\Mtech AI KU\deeplearningproject\Langgraph\langbho\lib\site-packages\langchain_core\language_models\base.py:328: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))
e:\Mtech AI KU\deeplearningproject\Langgraph\langbho\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anupd\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Deve

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [37]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I'm not sure! I don't have any information about your preferences. Would you like to tell me about your favorite ice cream flavors?"

In [38]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me 2 + 2.'

In [39]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [40]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't mention your name earlier. I'm happy to chat with you, though!"

In [41]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You didn't ask a math problem previously in our conversation. Our conversation started with you asking your name, and I told you I didn't have that information."